# Introduction

This notebook is aiming to broadly introduce all the major components of the toolsforexperiments suite.
the document is currently still work in progress, but already a decent starting point for a few things.

- Prerequisites: 
  - you should know the qcodes basics
  - have a look at the 'dummy_measurement_without_server' example before

# Imports

We need to have some common packages installed, and also need qcodes and plottr.
Refer to the official qcodes documentation for how to set up the right environment.

Plottr can be installed from pip or conda, or from github.

instrumentserver needs to be installed from github (and preferably using `pip install -e`).

In [ ]:
from pprint import pprint
import time

import numpy as np
from matplotlib import pyplot as plt
import h5py

from qcodes import Instrument, Station, find_or_create_instrument
from plottr.data import datadict_storage as dds, datadict as dd

# The instrument server

The role of the instrument server is to manage access to instruments (both real and virtual), so they can be used from multiple processes. 
It's not strictly necessary to use it, and its designed such that using instruments through the server can be used also to talk to instruments that are instantiated directly.

Start the instrumentserver from the commandline (in an env't that has qcodes and the instrumentserver package installed) in a directory that will be the working directory for the server.

```bash
    $ instrumentserver --gui True
```

Because no instruments are instantiated at this point, we get an empty window reflecting the blank qcodes station living inside the server.
We can already create a client in this notebook that connects to the instrumentserver:

In [ ]:
from instrumentserver.client import Client as InstrumentClient
cli = InstrumentClient() # connect to default host (localhost) and default port (5555)

We will now see how we can make use of the fact that we can talk to the instrumentserver from multiple processes. 
We'll create a virtual instrument (something with no direct counterpart on the hardware side) that allows us to manage arbitrary experiment parameters that we can access across the setup (a fancy way to keep track of important variables). After that, we'll set up some dummy instruments and perform some fake data acquisition with them.

## Parameter manager

We'll first launch the *parameter manager*. Because it has a dedicated GUI we run it as a standalone program. 
From a command line prompt we can launch it separately (again, make sure to do this within an env't that has the instrumentserver package installed):

```bash
    $ instrumentserver-param-manager
```

We'll simply get an empty window now.
The bottom of the window allows us to add arbitrary parameters and values, where dots serve as hierarchy separators (like objects and their children in python).
Let's add a few parameters, let's say `qubit.frequency`, `qubit.pipulse.len`, `qubit.pipulse.amp` and give them some arbitrary values.
We'll see something like this:

![Parameter manager window](figures/parammgr.png)

Because parameters live in the instrumentserver we can access the values here by creating a 'Proxy' instrument that forwards all requests to the 'real' instrument.

In [ ]:
params = cli.get_instrument('parameter_manager') # 'parameter-=_manager' is the name the startup script gives the instrument by default

# simply output the value of the pi pulse length:
params.qubit.pipulse.len()

The different clients (such as the one in the notebook kernel and the one of the GUI) should stay in sync automatically.
If we update a value from there, the GUI will display the change right away.
In turn, calling a parameter get function should always return the most value the server has.

In [ ]:
# after executing, the change is immediately visible in the GUI.
params.qubit.pipulse.len(100)

In [ ]:
# if we change the value in the GUI, this will return the new value.
params.qubit.pipulse.len()

The same goes for adding new parameters after all clients are connected.
We can add new parameters very easily from any client.
The GUI should update automatically once we add something here.

In [ ]:
# this will only be available in the GUI after pressing the refresh button.
params.add_parameter('qubit.anharmonicity', initial_value=-150.0, unit='MHz')

If we add a parameter in the instrument GUI (let's say: `resonator.frequency`), we can immediately access it from within the notebook.
Note, however, that autocomplete will not work right away (the client in the notebook needs to refresh, which only happens once we try to execute the command).

In [ ]:
params.resonator.frequency()

## Adding instruments to the server

When using 'vanilla' qcodes, we would simply create a qcodes station in our working kernel, then add instruments by instantiating the corresponding driver classes.
When using the instrumentserver to manage our instrument instances, we instead want to instantiate inside the station that the server runs.

As an example, we will add a dummy instrument that behaves like a vector network analyzer and simulates a resonator response.
Only the server will instantiate it, so instead of importing the driver, we pass the import path (of course, needs to be known in the environment that the server is running in).

In [ ]:
vna = cli.create_instrument(
    'instrumentserver.testing.dummy_instruments.rf.ResonatorResponse', # the path to the driver; must be importable for the server program.
    'vna',  # this is the name of the instrument in the server station
    f0=6.789e9,  # the resonance frequency of the mock resonator
    df=10e6,  # the linewidth
)

In [ ]:
# setting some reasonable values
vna.start_frequency(6.089e9)
vna.stop_frequency(7.089e9)
vna.npoints(1201)
vna.bandwidth(1e4)
vna.power(-50)

## Using the parameter manager in measurements

Refer to [measurement_notebook](https://github.com/toolsforexperiments/recipes/blob/master/dummy_measurement_without_server/measurement_notebook.ipynb) for a more in depth guide example of the measurement we are taking. 

The idea here is simple: by using parameters from the parameter manager we don't need to store configuration variables in measurement scripts, and can run functions using user-changeable parameters without having to pass variables. 
We illustrate this with an artificial, but also not too unrealistic dummy case: 
Assume we often want to sweep a flux (for instance for tuning a resonator) a small range around a given setpoint that can change.
Not having to keep track of that setpoint throughout scripts can make life a bit easier (and will hopefully lead to fewer mistakes).

First, lets instantiate the virtual flux controller, and define the measurement function:

In [ ]:
fluxctrl = cli.create_instrument(
    instrument_class='instrumentserver.testing.dummy_instruments.rf.FluxControl',
    name='fluxctrl',
    resonator_instrument='vna'
)

def sweep_flux(sweep_range=0.5, nsteps=51):
    flux, frequency, s11 = [], [], []
    
    vna.resonator_linewidth(10e6)
    vna.resonator_frequency(params.resonator.frequency() * 1e9)
    f0 = params.resonator.flux()-sweep_range/2.
    f1 = params.resonator.flux()+sweep_range/2.
    for f in np.linspace(f0, f1, nsteps):
        fluxctrl.flux(f)
        flux.append(f)
        frequency.append(vna.frequency())
        s11.append(vna.data())

    fig, ax = plt.subplots(1, 1, constrained_layout=True)
    ax.imshow(np.angle(np.array(s11)), aspect='auto', origin='lower', 
              extent=[frequency[0].min(), frequency[0].max(), f0, f1])

Let's add the parameter we want to control, ``resonator.flux``, via the GUI.
After that we can measure by setting parameters in the GUI, rather than typing them explicitly.

In [ ]:
# change resonator frequency or flux in the gui, and re-run to see the change.
sweep_flux()

It is of course clear that in such a simple experiment the Parameter Manager GUI provides only a limited amount of benefit.
However, once parameters are used across many different measurements, it can be very convenient to have it act as the central "single source of truth".

## Saving and recalling the state of the setup

Saving all current parameters in the instrumentserver to a file:

In [ ]:
cli.paramsToFile('parameters.json')

We can also save the state by pressing the `Save to file` button in the top left of server GUI. 
We can also utilize the `Save parameters to file` button from the parameter manager GUI to only save the parameters of the manager and not the entire state of the station.

By default `paramsToFile` saves only the name and value of a parameter, but we can also save units, for example:

In [ ]:
cli.paramsToFile('parameters-with-units.json', instrument='parameter_manager', attrs=['value', 'unit'])

Loading works similar:

In [ ]:
cli.paramsFromFile('parameters.json')

We can save and recall also from the GUI.
The instrumentserver GUI allows saving/loading the state of all instruments using a default file in the working directory (``parameters.json``).
For more control we can use the API as shown above.

In addition, the parameter manager GUI also allows saving/restoring all its parameters from the GUI. It saves to a separate file in the current working directory from where the parameter manager was launched (``parameter_manager-<name_of_instrument>.json``) and includes the units.

# Plottr: data management and inspection

Work in progress :)


For now, please check these resources:

* [Simple Measurement example](https://github.com/toolsforexperiments/recipes/blob/master/dummy_measurement_without_server/measurement_notebook.ipynb) - "dummy_measurement_without_server" example notebook
* [Using plottr with qcodes](https://qcodes.github.io/Qcodes/examples/plotting/How-to-use-Plottr-with-QCoDeS-for-live-plotting.html)

# Labcore: tools for increasing efficiency

Work in progress :)